In [1]:
#importing libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#initializing yolo algorithm
yolo=cv.dnn.readNet("D:\programming languages\OpenCV\YOLO\yolov3.weights","D:\programming languages\OpenCV\YOLO\yolov3.cfg")

In [3]:
#storing the class names in classes
classes=[]
with open("D:\programming languages\OpenCV\YOLO\coco.names","r") as f:
    classes=f.read().splitlines()
classes[:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

In [61]:
#distance between two points
def get_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

In [72]:
#threshold distance
distance_threshold=50

In [83]:
def start_video(path=0):
    # capturing the video
    capture=cv.VideoCapture(r"C:\Users\Sharath chandra\Downloads\people walking 2.webm")
    while True:
        isTrue,frame=capture.read()
        #isTrue -> True if frame is read else False
        if isTrue:
            #creating blob for an image
            blob=cv.dnn.blobFromImage(frame,1/255,(416,416),(0,0,0),swapRB=True,crop=False)
            #initially image will be read in BGR so swapRB=True will convert image to RGB
            #1/255  --> scaling an image
            #(416,416) size that can be accepted by our yolo model (yoloV3-416)
            yolo.setInput(blob)
            output_layers_names=yolo.getUnconnectedOutLayersNames()
            outputs=yolo.forward(output_layers_names)
            boxes=[]
            confidences=[]
            class_ids=[]
            centers=[]
            for output in outputs:
                for detection in output:
                    scores=detection[5:]
                    class_id=np.argmax(scores)
                    confidence=scores[class_id]
                    center_x=int(detection[0]*frame.shape[1])
                    center_y=int(detection[1]*frame.shape[0])
                    w=int(detection[2]*frame.shape[1])
                    h=int(detection[3]*frame.shape[0])
                    x=int(center_x-w/2)
                    y=int(center_y-h/2)
                    boxes.append([x,y,w,h])
                    confidences.append(confidence)
                    class_ids.append(class_id)
                    centers.append((center_x,center_y))
            indices=cv.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
            voilations=set()
            for  i in range(len(indices)-1):
                for j in range(i+1,len(indices)):
                    if get_distance(centers[indices[i]],centers[indices[j]])<distance_threshold:
                        voilations.add(indices[i])
                        voilations.add(indices[j])
            if len(indices)>0:
                for i in indices.flatten():
                    if class_ids[i]==0:
                        x,y,w,h=boxes[i]
                        if i in voilations:
                            cv.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                        else:
                            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv.imshow("Video",frame)
            if cv.waitKey(1) & 0xFF==ord('q'):
                break
        else:
            break
    capture.release()
    cv.destroyAllWindows()

In [84]:
start_video(r"C:\Users\Sharath chandra\Downloads\people walking 2.webm")